
### All the necessary packages to be imported

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os

### Uncomment the line which you want to use.

In [3]:
# Mainnet
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# Euljiro
# icon_service = IconService(HTTPProvider("https://test-ctz.solidwallet.io",3))
# Yeouido
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
# Pagoda
# icon_service = IconService(HTTPProvider("https://zicon.net.solidwallet.io", 3))
# Custom
# icon_service = IconService(HTTPProvider("http://54.186.60.134:9000/", 3))

### NIDs

In [4]:
# Mainnet
# NID = 1
# Euljiro Testnet
# NID = 2
# Yeouido Testnet // Tbears
NID = 3
# Pagoda Testnet
# NID = 80

### List necessary SCORE addresses in this section

In [44]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Yeouido Testnet
SCORE_ADDRESS = "cx2a0e0737702b894af1e03442a6c8838f6aa80ab0"


# Mainnet
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

### Import the necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [57]:
deployer_wallet = KeyWallet.load(<path to keystore file>,<password>)
caller_wallet = KeyWallet.load(<path to keystore file>,<password>)

SyntaxError: invalid syntax (<ipython-input-57-138e1b62e38d>, line 1)

### Create new wallets 

If you want to use new wallets, execute the cell below:-

In [41]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

hx3fd86f0c4c7761c030daa5d682e869fd7279eb7d
hxc9ef39e8294fa20a38a878e16bf7d8e526c51cf5


### Load ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [43]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('dice.zip'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xfd069c0fc42680b62b8591db817663d2b45248653c6553f706c36cf715cc6213',
 'blockHeight': 2175313,
 'blockHash': '0xeca5cd6af5ee5ee572a5f304088fb9a87fa1283fa81a00aaa318b36427021c36',
 'txIndex': 1,
 'to': 'cx0000000000000000000000000000000000000000',
 'scoreAddress': 'cx2a0e0737702b894af1e03442a6c8838f6aa80ab0',
 'stepUsed': 1083405920,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1083405920,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

### Updating contract

In [40]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('dice.zip'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32011, 'message': 'Out of balance: balance(5570050000000000000) < value(0) + fee(16835056000000000000)'}


JSONRPCException: {'code': -32011, 'message': 'Out of balance: balance(5570050000000000000) < value(0) + fee(16835056000000000000)'} (Json rpc error)

### Calling readonly methods for values

In [45]:
_call = CallBuilder()\
    .from_(caller_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .method("get_game_status")\
    .build()
response = icon_service.call(_call)
print(response)
   

0x0


### Calling external methods

In [46]:
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS) \
    .nid(NID) \
    .nonce(100) \
    .method("toggle_game_status")\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx3fd86f0c4c7761c030daa5d682e869fd7279eb7d', 'to': 'cx2a0e0737702b894af1e03442a6c8838f6aa80ab0', 'stepLimit': '0x38770', 'timestamp': '0x5a69d8fa59fca', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'toggle_game_status'}, 'signature': '8fmXdMQXvAiS7Lw4Pu541SYDeGu2bAMY7IBXqZA2PcUXNvqMV5Aam+5xcbB4aTDMol/xOvH/Rn9s3VCkaYazeAA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5953a64b716ccefe5eafcbe097b141d8d8ebe18121254f7aa1ac3aa7b8b83250',
 'blockHeight': 2175333,
 'blockHash': '0xdfdd8227812b7352293aadf0dd26284951e669ddbf1092c7c43036ffb7de7c42',
 'txIndex': 1,
 'to': 'cx2a0e0737702b894af1e03442a6c8838f6aa80ab0',
 'stepUsed': 131280,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 131280,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

### ICX Transfer

In [49]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID) \
    .nonce(100) \
    .value(50*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx3fd86f0c4c7761c030daa5d682e869fd7279eb7d', 'to': 'cx2a0e0737702b894af1e03442a6c8838f6aa80ab0', 'stepLimit': '0x36ee8', 'timestamp': '0x5a69d922408aa', 'nid': '0x3', 'value': '0x2b5e3af16b1880000', 'nonce': '0x64', 'signature': '+uY6cBkDXReNfskC0u1PhF3RsRZsS3kctj6UufOwENglzGMj0ryQqSkzujWKZeA59SUaBp1EgKbe2pwiOBk8UQA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x1a8d3c98ea20c5b2b8dc16a292eb55ac71a8946d6803fffdd45f8964761280a0',
 'blockHeight': 2175354,
 'blockHash': '0x82ea75b14c5310ffeed8de4e3e40c6d1d63b096cc835b7dec9a64757298ebc9e',
 'txIndex': 1,
 'to': 'cx2a0e0737702b894af1e03442a6c8838f6aa80ab0',
 'stepUsed': 125000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 125000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [50]:
balance = icon_service.get_balance(SCORE_ADDRESS)
print(balance)

50000000000000000000


### Calling payable methods

In [56]:
params={
    "upper": 99,
    "lower": 1,
    "user_seed": "Lucky",
    "side_bet_amount": 10**18,
    "side_bet_type":"icon_logo1"
}
# Skip side_bet_amount and side_bet_type if you dont want to play sidebet

call_transaction = CallTransactionBuilder()\
    .from_(caller_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .value(10*10**18)\
    .method("call_bet")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, caller_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32032, 'message': 'Invalid gap. Choose upper and lower values such that gap is between 0 to 95'}


JSONRPCException: {'code': -32032, 'message': 'Invalid gap. Choose upper and lower values such that gap is between 0 to 95'} (Json rpc error)